## Redes Neurais e Aprendizado Profundo

#### Redes Densas em pytorch e detalhes:
##### - camadas pré-implementadas
##### - opções para acessar parametros
##### - uso do gradiente
##### - backpropagation e algoritmos de otimização

Moacir A Ponti - 2022

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Devemos definir a função `forward` e `backward` (onde os gradientes são computados) é definida usando o autograd. Podemos usar qualquer operador de tensor na função `forward`

In [ ]:
class Network(nn.Module):

  def __init__(self):
    # fazer a inicializacao conforme a superclasse nn.Module
    super(Network, self).__init__()
    # cria o projeto da rede neural
    # imagens de 28x28 pixels = 784 valores
    # saída da rede = 10 classes
    self.fc1 = nn.Linear(784, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    # imagem 28x28 pixels -> nao funciona em camada densa (1d)
    # recebe minibatches
    x = torch.flatten(x, 1) # achata dimensoes menos a dos batches
    x = F.relu(self.fc1(x)) # camada densa + relu
    x = self.fc2(x) # camada densa linear
    return x


In [ ]:
net = Network()
print(net)

Network(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)


In [ ]:
input_random = torch.randn(1, 1, 28, 28)
print(input_random.shape)
output = net(input_random)
print(output)

torch.Size([1, 1, 28, 28])
tensor([[-0.0229, -0.0728,  0.2644, -0.1998,  0.0665,  0.4571,  0.0812,  0.0353,
         -0.2039, -0.5764]], grad_fn=<AddmmBackward0>)


É possível acessar os parametros da rede

In [ ]:
params = list(net.parameters())
print(len(params))
# primeira camada:
print(params[0].size())

4
torch.Size([32, 784])


Backpropagation: `backward`

In [ ]:
# zerar o buffer de gradientes
net.zero_grad()
output.backward(torch.randn(1,10))

In [ ]:
output = net(input_random)
target = torch.randn(10) # target aleatorio como exemplo
print(target)
target = target.view(1, -1)
print(target)

criterion = nn.MSELoss() # mean squared error
loss = criterion(output, target)
print(loss)

tensor([ 2.0284, -0.4078, -0.2471,  0.2618, -1.1634,  1.4772,  0.5466, -1.1262,
         0.7802,  2.1103])
tensor([[ 2.0284, -0.4078, -0.2471,  0.2618, -1.1634,  1.4772,  0.5466, -1.1262,
          0.7802,  2.1103]])
tensor(1.7101, grad_fn=<MseLossBackward0>)


In [ ]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) # funcao linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # relu

Backpropagation com `backward`

In [ ]:
net.zero_grad()

print('antes')
print(net.fc1.bias.grad)

loss.backward()
print('depois')
print(net.fc1.bias.grad)


antes
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
depois
tensor([-0.0697,  0.0000,  0.0000,  0.0780,  0.0607,  0.0000,  0.0777, -0.1289,
         0.0000,  0.0000,  0.0000,  0.1562,  0.0147, -0.1293,  0.0000,  0.1269,
         0.0000,  0.0000,  0.0420,  0.1020,  0.0048,  0.0000,  0.0000,  0.0000,
         0.0000, -0.0815, -0.0008,  0.0000,  0.0000,  0.0000, -0.0559,  0.0492])


Adptacao dos parametros da rede

- otimizador: SGD, Adam, etc.

In [ ]:
print(params[0][0][:10])

tensor([-0.0319,  0.0181,  0.0137,  0.0292, -0.0176,  0.0059, -0.0271,  0.0273,
         0.0109, -0.0266], grad_fn=<SliceBackward0>)


In [ ]:
import torch.optim as optim

# cria o objeto otimizador
optimizer = optim.SGD(net.parameters(), lr=0.01)

# para cada loop de treinamento

## zerar o buffer de gradientes
optimizer.zero_grad()
## gerar a saída e computar gradientes com relacao à funcao de perda
output = net(input_random)
loss = criterion(output, target)
loss.backward()
## realizar a adaptacao dos pesos
optimizer.step()

In [ ]:
print(params[0][0][:10])

tensor([-0.0324,  0.0183,  0.0137,  0.0284, -0.0184,  0.0057, -0.0280,  0.0260,
         0.0111, -0.0272], grad_fn=<SliceBackward0>)
